<a href="https://colab.research.google.com/github/thomasonzhou/CS443_Implementations/blob/master/thomsCodeColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.chdir("/content/drive/My Drive/fingerprints_combined")
# os.listdir()

Mounted at /content/drive


In [30]:
import os
import matplotlib.pyplot as plt

IMG_DIR = "/content/drive/My Drive/fingerprints_combined"


def print_data_info():
    files = os.listdir(IMG_DIR)
    fvc_names = [name for name in files if "FVC" in name]
    uareu_names = [name for name in files if "UareU" in name]
    fv_system_names = [name for name in files if "fingerprint" in name]
    print(f"{len(fvc_names)/8} Fingerprint verification competition")
    print(f"{len(uareu_names)/8} U.are.U")
    print(f"{len(fv_system_names)/8} Fingerprint verification system")
    print(f"{len(files)} Files")


def show_visualization_samples(dataset, count=4):
    fig = plt.figure()

    for i, (sample1, sample2, same_label) in enumerate(dataset):

        ax = plt.subplot(2, count, i + 1)
        # plt.tight_layout()
        ax.set_title(f'{same_label[0]}')
        ax.axis('off')

        # change to black and white
        plt.imshow(sample1.squeeze(), cmap="gray")
        # plt.imshow(sample.permute(1, 2, 0))

        ax = plt.subplot(2, count, i + 1 + count)
        plt.imshow(sample2.squeeze(), cmap="gray")
        ax.axis('off')

        if i == count - 1:
            plt.show()
            break


if __name__ == "__main__":
    print_data_info()


1320.0 Fingerprint verification competition
65.0 U.are.U
21.0 Fingerprint verification system
11248 Files


In [31]:
import pandas as pd
import torch
from torch.utils.data import Dataset
import os
from PIL import Image


class FingerprintDataset(Dataset):

    #script_dir = os.path.dirname(__file__)
    script_dir = os.path.dirname(IMG_DIR)

    def __init__(self, train=True, transform=None):
        file_path = os.path.join(self.script_dir, "metadata.csv")
        self.csv = pd.read_csv(file_path)

        if train:
            self.csv = self.csv[self.csv["test_set"] == False]
        else:
            self.csv = self.csv[self.csv["test_set"] == True]#removes condition for all se
        self.transform = transform
        # print(self.csv.columns)
        # ['dataset', 'original_filename', 'person_id', 'impression_id', 'finger_id', 'fileextension', 'db_id', 'filename', 'test_set']

    def __len__(self):
        return len(self.csv)
        return

    def __getitem__(self, idx):

        if torch.is_tensor(idx):
            idx = idx.to_list()

        # potentially invert the image

        get_same_class = torch.rand(1) < 0.5

        img1_name, img2_name = self.get_pair_names(idx, get_same_class)

        img1_path = os.path.join("./fingerprints_combined", img1_name)
        img2_path = os.path.join("./fingerprints_combined", img2_name)

        # L is for black and white, RGB is for color
        image1 = Image.open(img1_path).convert("L")
        image2 = Image.open(img2_path).convert("L")

        if self.transform:
            image1 = self.transform(image1)
            image2 = self.transform(image2)

        # print(get_same_class)
        # print(self.csv.iloc[idx, -2], class_sample.iloc[0, -2])

        label = 1 if get_same_class else 0
        return image1, image2, label

    def get_pair_names(self, idx, get_same_class=True):

        img1_name = self.csv.iloc[idx, -2]

        person_id = self.csv.iloc[idx, 2]

        # indexing for impressions, UareU has an extra column
        finger_idx = 4 if self.csv.iloc[idx, 0] == "UareU" else 2
        finger_id = self.csv.iloc[idx, finger_idx]

        # don't compare the same file with itself
        different_file = self.csv.loc[(self.csv.iloc[:, -2] != img1_name)]

        if get_same_class:
            # FVC2000, 2002, 2004 are different
            same_dataset = different_file[(
                different_file.iloc[:, 0] == self.csv.iloc[idx, 0])]

            # DB 1, 2, 3, 4 are different for FVC (additional processing)
            if "FVC" in self.csv.iloc[idx, 0]:

                # get the DB number from the filename
                # e.g. FVC2002-DB1_A-50_6.tif
                db_id = self.csv.iloc[idx, -3]

                same_dataset = same_dataset[(
                    same_dataset.iloc[:, -3] == db_id)]

            same_finger = same_dataset[(
                same_dataset.iloc[:, finger_idx] == finger_id)]

            if self.csv.iloc[idx, 0] == "UareU":
                # filter by person id
                same_finger = same_finger.loc[(
                    same_finger.iloc[:, 2] == person_id)]

            # print(f"same_count {len(same_finger)}")
            class_sample = same_finger.sample(n=1)
        else:
            different_class = different_file[(
                different_file.iloc[:, finger_idx] != finger_id)]

            if self.csv.iloc[idx, 0] == "UareU":
                # filter by person id
                different_class = different_class[(
                    self.csv.iloc[:, 2] != person_id)]

            # print(f"different_count {len(different_class)}")
            class_sample = different_class.sample(n=1)

        img2_name = class_sample.iloc[0, -2]

        return img1_name, img2_name



In [32]:
import sys
sys.path.append("../")
#from data_loading import FingerprintDataset
import pandas as pd
import os

"""
This script needs to be run from the model_evaluation folder.
"""

#assert os.path.isfile("./testset.csv") == False, "File already exists, do not overwrite."

# generate new csv with test set, equal number of same and different fingerprint pairs
# generate 1000 pairs, 500 same, 500 different
NUMBER_OF_PAIRS = 1000
HALF_PAIRS = NUMBER_OF_PAIRS // 2

test_data = FingerprintDataset(train=False)
print(f"Test data: {len(test_data)}")


# test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=True)

df = pd.DataFrame(columns=["img1_name", "img2_name", "same_class"])

for i in range(HALF_PAIRS):
    image1, image2, = test_data.get_pair_names(i, get_same_class=True)
    df.loc[i] = [image1, image2, 1]
for i in range(HALF_PAIRS):
    image1, image2, = test_data.get_pair_names(i, get_same_class=False)
    df.loc[i + HALF_PAIRS] = [image1, image2, 0]


df.to_csv("./testset.csv", index=False)


Test data: 960


In [33]:
import torch


class ContrastiveLoss(torch.nn.Module):
    """
    Given the outputs of two identical sister networks, return the loss

    References:
    https://github.com/delijati/pytorch-siamese/blob/master/contrastive.py
    https://hackernoon.com/facial-similarity-with-siamese-networks-in-pytorch-9642aa9db2f7?ref=hackernoon.com
    """

    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def check_valid_inputs():
        pass

    def forward(self, x1, x2, y):

        # euclidean distance
        diff = x1 - x2
        dist_sq = torch.sum(torch.square(diff), dim=1)
        dist = torch.sqrt(dist_sq)
        # assert dist == torch.cdist(x1, x2)

        mdist = self.margin - dist
        clamp_mdist = torch.clamp(mdist, min=0)
        clamp_mdist_sq = torch.square(clamp_mdist)

        loss = (1-y)*0.5*dist_sq + y*0.5*clamp_mdist_sq
        return torch.mean(loss)


In [34]:
import sys
sys.path.append("../")
#from data_loading import FingerprintDataset
import pandas as pd
import os

"""
This script needs to be run from the model_evaluation folder.
"""

#assert os.path.isfile("./testset.csv") == False, "File already exists, do not overwrite."

# generate new csv with test set, equal number of same and different fingerprint pairs
# generate 1000 pairs, 500 same, 500 different
NUMBER_OF_PAIRS = 1000
HALF_PAIRS = NUMBER_OF_PAIRS // 2

test_data = FingerprintDataset(train=False)
print(f"Test data: {len(test_data)}")

# test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=True)

df = pd.DataFrame(columns=["img1_name", "img2_name", "same_class"])

for i in range(HALF_PAIRS):
    image1, image2, = test_data.get_pair_names(i, get_same_class=True)
    df.loc[i] = [image1, image2, 1]
for i in range(HALF_PAIRS):
    image1, image2, = test_data.get_pair_names(i, get_same_class=False)
    df.loc[i + HALF_PAIRS] = [image1, image2, 0]


df.to_csv("./testset.csv", index=False)


Test data: 960


In [35]:
import torch.nn as nn
class TorchSiameseNetwork(nn.Module):
    def __init__(self):
        super(TorchSiameseNetwork, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 4, kernel_size=3, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(4, 8, kernel_size=3, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(8, 16, kernel_size=3, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
        )
        self.mlp = nn.Sequential(
            nn.Linear(512, 500),
            nn.ReLU(inplace=True),
            nn.Linear(500, 500),
            nn.ReLU(inplace=True),
            nn.Linear(500, 5),
        )

    def forward_once(self, x):
        output = self.cnn(x)
        output = output.view(output.size()[0], -1)
        output = self.mlp(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2


In [36]:
import torch.nn as nn

class TorchVGG(nn.Module):
    def __init__(self):
        super(TorchVGG, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2)
        )
        self.mlp = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, 5)
        )

    def forward_once(self, x):
        output = self.cnn(x)
        output = output.view(output.size()[0], -1)
        output = self.mlp(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

In [37]:
from torchvision import transforms
#from data_loading import FingerprintDataset, show_visualization_samples
from torch.utils.data import DataLoader
#from models import ContrastiveLoss, TorchVGG
from torch.optim import Adam
#import my_transforms
import torch
import matplotlib.pyplot as plt
import os
from torchvision import transforms


transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor()
])

train_data = FingerprintDataset(train=True, transform=transform)
test_data = FingerprintDataset(train=False, transform=transform)

print(f"Train data: {len(train_data)}")
print(f"Test data: {len(test_data)}")
# show_visualization_samples(test_data, count=5)

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)

# model = TorchSiameseNetwork()
model = TorchVGG()
# criterion = ContrastiveLoss(margin=1.0)
criterion = nn.functional.binary_cross_entropy_with_logits(preds, y.float())
optimizer = Adam(model.parameters(), lr=0.00001)

# Training

EPOCHS = 1
device = torch.device("cpu")

LOAD_SAVED_MODEL = True
SAVED_MODEL_PATH = "torch_cnn_transform3_lr22.pt"

losses = []

#if LOAD_SAVED_MODEL and os.path.exists(SAVED_MODEL_PATH):
#    model.load_state_dict(torch.load(SAVED_MODEL_PATH))
#    print("Loaded saved model")
for epoch in range(1, EPOCHS + 1):
    for i, (image1, image2, same_class) in enumerate(train_loader, start=1):
        image1, image2, same_class = image1.to(
            device), image2.to(device), same_class.to(device)

        optimizer.zero_grad()

        x1, x2 = model(image1, image2)

        loss = criterion(x1, x2, same_class)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch} iteration {i} loss: {loss}")
            losses.append(loss.item())

# rename original model and save as prev
#if os.path.isfile(SAVED_MODEL_PATH):
#    os.rename(SAVED_MODEL_PATH, "./models/saved_parameters/torch_cnn_transform3_lr22_prev.pt")

# save new model
torch.save(model.state_dict(),  SAVED_MODEL_PATH)

test_loader = DataLoader(test_data, batch_size=1, shuffle=True)
test_iter = iter(test_loader)
# image1, _, _ = next(test_iter)

# even number of samples please
SAMPLES = 20

fig, axes = plt.subplots(2, SAMPLES//2, figsize=(SAMPLES*2, 8))
axes = axes.flatten()
for i in range(SAMPLES):
    image1, image2, label = next(test_iter)
    concatenated = torch.cat((image1, image2), 3)

    image1, image2, label = image1.to(
        device), image2.to(device), label.to(device)

    x1, x2 = model(image1, image2)

    dist = torch.nn.functional.pairwise_distance(x1, x2)
    # loss = criterion(x1, x2, label)

    concatenated = concatenated.squeeze().cpu().numpy()

    ax = axes[i]
    ax.imshow(concatenated, cmap='gray')
    ax.set_title(
        f"Dist: {dist.item():.2f} {True if label.item() else False}")
    ax.axis('off')
plt.tight_layout()
plt.show()


Train data: 10288
Test data: 960


NameError: ignored

In [ ]:
import torch.nn as nn

class TorchVGG(torch.nn.Module):
    def __init__(self):
        super(TorchVGG, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2)
        )
        self.mlp = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128,  128)
        )

    def forward_once(self, x):
        output = self.cnn(x)
        output = output.view(output.size()[0], -1)
        output = self.mlp(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

In [101]:
# references:
# https://datahacker.rs/019-siamese-network-in-pytorch-with-application-to-face-similarity/
# https://github.com/pytorch/examples/blob/main/siamese_network/main.py

import torchvision

# create the Siamese Neural Network
class SiameseNetwork_MobileNet(nn.Module):

    def __init__(self):
        super(SiameseNetwork_MobileNet, self).__init__()

        # load mobile net architecture (not pre-trained weights)
        self.feature_extractor = torchvision.models.mobilenet_v3_small()
        # modify input layer
        self.feature_extractor.features[0][0] = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        # modify output layer
        self.feature_extractor.classifier[3] = nn.Linear(1024, 256)

        # classifier layer to predict if same finger or not
        self.fc =  nn.Sequential(
            nn.Linear(256,1)
        )

    def forward_once(self, x):
        # This function will be called for both images
        # Its output is used to determine the similiarity
        x = x.unsqueeze(1)
        output = self.feature_extractor(x)

        return output

    def forward(self, input1, input2):
        # In this function we pass in both images and obtain both vectors
        # which are returned

        # print(f"Input1 shape before forward_once: {input1.shape}")
        # print(f"Input2 shape before forward_once: {input2.shape}")
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        # print(output1)

        # get absolute difference of extracted features
        output = torch.abs(output1 - output2)
        output = self.fc(output).view(-1)

        # return output1, output2
        return output

In [ ]:
# new implementation with Pytorch Lightning, Weights and biases library
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning
!pip install wandb

In [109]:
import wandb
from lightning.pytorch.loggers import WandbLogger
# wandb.finish()

wandb_config = {
    "architecture": "Simple_CNN",
    "datasets": "all",
    "loss": "binary_cross_entropy"
}

# hyperparameters tbd

wandb_logger = WandbLogger(project="siamese_fingerprint", config = wandb_config)
# wandb.init(
#     project="siamese-fingerprint",
#     config = wandb_config

# )

In [105]:
import lightning.pytorch as pl

class CustomModule(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def get_acc(self, preds, labels):
        output = (preds > 0.5).float()
        correct = (output == labels).float().sum()

        return correct / preds.shape[0]

    def get_eucledian_acc(self, preds, labels):
        output = (preds > 0.15).float()
        correct = (output == labels).float().sum()

        return correct / preds.shape[0]

    def training_step(self, batch, batch_idx):
        x1, x2, y = batch
        preds = self.model(x1, x2)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(preds, y.float())
        self.train_acc = self.get_acc(torch.sigmoid(preds), y)

        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", self.train_acc, prog_bar=True, on_step=False, on_epoch=True, logger=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x1, x2, y = batch
        preds = self.model(x1, x2)

        # print(preds, y)
        loss = nn.functional.binary_cross_entropy_with_logits(preds, y.float())
        self.val_acc = self.get_acc(torch.sigmoid(preds), y)

        self.val_loss = loss

        self.log("val_loss", self.val_loss, prog_bar=True)
        self.log("val_acc", self.val_acc, prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):
        return self.validation_step(batch, batch_idx)

    def predict_step(self, batch, batch_idx):
        x1, x2 = batch
        preds = self.model(x1, x2)

        return torch.sigmoid(preds)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, factor=0.5, mode='min', verbose=True, patience=15, threshold=0)
        return {
           'optimizer': optimizer,
           "lr_scheduler": {
                "scheduler": lr_scheduler,
                "monitor": "val_loss"}
       }


vgg = TorchVGG()

VGG_BCE = CustomModule(vgg)

mobile = SiameseNetwork_MobileNet()
litmodel = CustomModule(mobile)



In [106]:
import enum
from torch.utils.data import Dataset
import os
import pandas as pd
import torch
from PIL import Image
import numpy as np

class DatasetType(enum.Enum):
    TRAIN = 0
    VAL = 1
    TEST = 2

class StaticFingerprintDataset(Dataset):
    def __init__(self, dataset_type: DatasetType, transform=None):
        self.dataset_type = dataset_type
        self.transform = transform

        # Load testset.csv, trainset.csv, or valset.csv

        if self.dataset_type == DatasetType.TRAIN:
            file_path = "/content/drive/My Drive/trainset.csv"
        elif self.dataset_type == DatasetType.VAL:
            file_path = "/content/drive/My Drive/valset.csv"
        elif self.dataset_type == DatasetType.TEST:
            file_path = "/content/drive/My Drive/testset.csv"

        self.csv = pd.read_csv(file_path)
        # print(self.csv.columns)

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.to_list()

        img1_name = self.csv.iloc[idx, 0]
        img2_name = self.csv.iloc[idx, 1]
        label = self.csv.iloc[idx, 2]

        img1_path = os.path.join("/content/drive/My Drive/fingerprints_combined", img1_name)
        img2_path = os.path.join("/content/drive/My Drive/fingerprints_combined", img2_name)

        # L is for black and white, RGB is for color
        image1 = Image.open(img1_path).convert("L")
        image2 = Image.open(img2_path).convert("L")

        image1 = image1.resize((100, 100))
        image2 = image2.resize((100, 100))

        # Convert images to numpy arrays and change datatype to float32
        image1 = np.array(image1, dtype=np.float32)
        image2 = np.array(image2, dtype=np.float32)

        label = self.csv.iloc[idx, 2]
        # print(label)

        # if self.transform:
        #     image1 = self.transform(image1)
        #     image2 = self.transform(image2)

        return image1, image2, label

transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor()
])

train_data = StaticFingerprintDataset(DatasetType.TRAIN)
val_data = StaticFingerprintDataset(DatasetType.VAL)
test_data = StaticFingerprintDataset(DatasetType.TEST)

# train_data = StaticFingerprintDataset(DatasetType.TRAIN, transform=transform)
# val_data = StaticFingerprintDataset(DatasetType.VAL, transform=transform)
# test_data = StaticFingerprintDataset(DatasetType.TEST, transform=transform)

BATCH_SIZE = 1

train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


In [ ]:
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint

early_stop_callback = EarlyStopping(
   monitor='val_loss',
   min_delta=0.00,
   patience=35,
   verbose=False,
   mode='min'
)

checkpoint_callback = ModelCheckpoint(
   dirpath="./checkpoints",
   monitor='val_loss',
   filename='best'
)

# wandb.init(project="siamese-fingerprint")

trainer = pl.Trainer(accelerator='gpu', max_epochs=3000,
                    #  logger=wandb_logger,
                     callbacks=[early_stop_callback, checkpoint_callback])
trainer.fit(model=litmodel, train_dataloaders=train_loader, val_dataloaders=val_loader)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name  | Type                     | Params
---------------------------------------------------
0 | model | SiameseNetwork_MobileNet | 1.8 M 
---------------------------------------------------
1.8 M     Trainable params
0         Non-trainable params
1.8 M     Total params
7.121     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.mod

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

1
11

1
1
1
tensor([0.0171], device='cuda:0') tensor([1], device='cuda:0')
tensor([0.0171], device='cuda:0') tensor([1], device='cuda:0')


Training: |          | 0/? [00:00<?, ?it/s]

1
0
0
0
1
1
1
0
1
1
1
0
0
0
1
1
0
0
1
0
0
0
1
1
0
1
1
1
1
0
0
1
1
1
1
1
1
0
1
0
1
1
0
0
0
1
0
0
1
0
1
1
1
1
0
0
0
1
1
0
1
0
1
1
0
1
0
0
0
1
0
1
0
0
1
0
1
0
0
0
1
1
